In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
%matplotlib inline
sns.set(font_scale=1.5);

### Logistic Regression Primer
https://youtu.be/yIYKR4sgzI8?t=181

In [3]:
glass = pd.read_csv('data/glass.csv',names=[
    'ri','na','mg','al','si','k','ca','ba','fe','glass_type'],skiprows=1)
# Glass Types 1, 2, 3 in are window glass.
# Types 5, 6, 7 are household glass.
glass['household'] = glass.glass_type.map({1:0, 2:0, 3:0, 5:1, 6:1, 7:1})
glass.head(1)

,ri,na,mg,al,si,k,ca,ba,fe,glass_type,household
0,1.52101,13.64,4.49,1.1,71.78,0.06,8.75,0.0,0.0,1,0


In [4]:
# instantiate classifier
logr = LogisticRegression(solver='lbfgs')
# set up predictor matrix
X = glass[['al']]
# set up target variable
y = glass.household
# fit classifier
logr.fit(X,y)

LogisticRegression()

## The logit function

$$\log \left({p\over 1-p}\right) = \beta_0 + \beta_1x$$

The intercept plus the coefficient times a value for `al` equals our log odds for a given observation.

### Primer on Log Odds
To understand how logistic regression predicts the probability of class membership we need to start by understanding the relationship between probability, odds ratios, and log odds ratios. This is because logistic regression predicts log odds and so reading log odds is necessary for interpreting logistic regression.

$$probability = \frac {one\ outcome} {all\ outcomes}$$

$$odds = \frac {one\ outcome} {all\ other\ outcomes}$$

In [ ]:
print('Dice Roll of 1')
print('probability: 1/6')
print ('odds: 1/5')
print ('log odds: '+str(np.log(1/5)))

In [ ]:
print('Even Dice Roll')
print('probability: 3/6')
print ('odds: 3/3')
print ('log odds: '+str(np.log(3/3)))

In [ ]:
# odds can never be negative but log odds can
print(np.log(5/2))
print(np.log(2/5))

In [ ]:
# we can exponentiate log odds to get back to odds
# from odds we can easily get back to probability
print(np.exp(np.log(2/5)))

### Interpreting logistic regression attributes
    * our logistic regression, like linear regression, has coefficients and an intercept
    * but in logistic regression, our coefficients and intercept are expressed in log odds

In [ ]:
print('logr intercept: '+str(logr.intercept_[0]))
print('al coef: '+str(logr.coef_[0][0]))

* when aluminimum equals zero, what are the log odds of household = 1?
* for every unit of aluminum gained, what is the log odds increase of household = 1?

** To really interpret our equation, we need to convert log odds to probabilities.**

In [ ]:
# function to convert log odds to probability
def log_to_prob(al_value):
    # add coef, aluminum value, intercept
    logodds = logr.coef_[0][0]*al_value+logr.intercept_[0]
    # exponentiate sum  - the inverse of taking the log - to get odds
    odds = np.exp(logodds)
    # convert odds to probability
    prob = odds/(1 + odds)
    # return probability
    return prob

In [ ]:
print('p value when al is 2.5: '+str(log_to_prob(2.5)))
print('p value when al is 0: '+str(log_to_prob(0)))
print('change in probability: '+str(log_to_prob(2.5)-log_to_prob(0)))

In [ ]:
# the predict proba method does this work for us
logr.predict_proba([[2.5]])[:,1][0]

In [ ]:
# Store the predicted probabilities of class 1 - below I'm taking all the rows and the second column
fig = plt.figure(figsize=(12,8))
glass['household_pred_prob'] = logr.predict_proba(X)[:, 1]
glass.sort_values('household_pred_prob',inplace=True)
# Plot the predicted probabilities.
plt.scatter(glass.al, glass.household)
plt.plot(glass.al, glass.household_pred_prob, color='red')
plt.xlabel('al')
plt.ylabel('household_proba')
plt.show()
# what's the probability of household = 1 when aluminum is 1.5? 2.0? 3.5?
# what about if we got an aluminiumum value of 6?

### Referencing the modeling framework, create a row for logistic regression in your own words


# The Confusion Matrix and Other Classification Metrics

$$Accuracy = \frac{total~predicted~correct}{total~predicted}$$

Accuracy alone doesn’t always give us a full picture.

If we know a model is 75% accurate, it doesn’t provide any insight into why the 25% was wrong.

Consider a binary classification problem where we have 165 observations/rows of people who are either smokers or nonsmokers.

- **True positives (TP):** These are cases in which we predicted yes (smokers), and they actually are smokers.
- **True negatives (TN):** We predicted no, and they are nonsmokers.
- **False positives (FP):** We predicted yes, but they were not actually smokers. (This is also known as a "Type I error.")
- **False negatives (FN):** We predicted no, but they are smokers. (This is also known as a "Type II error.")
<table style="border: none">
<tr style="border: none">
    <td style="border: none; vertical-align: bottom">n = 165</td>
    <td style=""><b>Predicted: No</b></td>
    <td style=""><b>Predicted: Yes</b></td>
</tr>
<tr>
    <td><b>Actual: No</b></td>
    <td style="text-align: center">TN = 50</td>
    <td style="text-align: center">FP = 10</td>
    <td style="text-align: center">60</td>
</tr>
<tr>
    <td><b>Actual: Yes</b></td>
    <td style="text-align: center">FN = 5</td>
    <td style="text-align: center">TP = 100</td>
    <td style="text-align: center">105</td>
</tr>
<tr style="border: none">
    <td style="border: none"></td>
    <td style="text-align: center">55</td>
    <td style="text-align: center">110</td>
</tr>

</table>

In [ ]:
from sklearn.metrics import accuracy_score
# high accuracy
preds = logr.predict(X)
accuracy_score(y,preds)

In [ ]:
from sklearn.metrics import confusion_matrix
conmat = confusion_matrix(y, preds)
confusion = pd.DataFrame(conmat, index=['Actual: 0','Actual: 1'],
                         columns=['Pred: 0','Pred: 1'])
# precision (TP/(TP+FP)) is good - we're correct 26 of 29 times we predicted a positive
# but middling recall (TP/(TP+FN)) - only 26 of 51 positive cases correct
# an f1 score is a harmonic mean of these two metrics
confusion

### there are cases where it's desirable to have a lower threshold than .5, typically where false negatives might be extremely undesirable, and we can reduce them by tolerating more false positives


In [ ]:
probas = logr.predict_proba(X)[:,1]
thresh = .4
# predict that any pred over thresh is a positive
preds = pd.Series(probas).map(lambda x: 1 if x>=thresh else 0)
# score in conmat
conmat = confusion_matrix(y, preds)
confusion = pd.DataFrame(conmat, index=['Actual: 0','Actual: 1'],
                         columns=['Pred: 0','Pred: 1'])
# at a lower threshold, we increase our predicted TPs by 6, as well as our FPs by 6
confusion